In [1]:
import requests
import os

In [2]:
DEV = False

if_dev = '.dev' if DEV else ''


settles_base_url = f'http://settles-api{if_dev}.mosaic.hartreepartners.com/settles/api/v1/'
tc_base_url = f'http://trader-curves-api{if_dev}.mosaic.hartreepartners.com/trader-curves/api/v1/'


In [3]:

import pandas as pd
from mosaic_sql.postgres import get_engine as get_postgres_engine

In [4]:
# postgres_engine = get_postgres_engine('ttda.postgres.storage.dev.mosaic.hartreepartners.com',
#                                       '5437','postgres','p0stgresisforttda','postgres')

postgres_prod_engine = get_postgres_engine('ttda.postgres.storage.mosaic.hartreepartners.com',
                                      '5437','postgres','p0stgrespr0d4ttda','postgres')

In [27]:
# df_dev = pd.read_sql("select * from settles.trader_curves where symbol like '%-TS' and date='2021-06-15'", con=postgres_engine)
df_prod = pd.read_sql("select * from settles.trader_curves where symbol like '%-TS' and date='2021-06-15'", con=postgres_prod_engine)
final_df = None
for idx,r in df_prod.iterrows():
    source = r['source']
    symbol=r['symbol']
    sql = f"""
    select concat(symbol,'#',source) symbol,date,epoch_time,definition->>'anchor_spread' as anchor_spread
    ,jsonb_array_elements_text(tc.definition->'contracts') as contracts ,
    jsonb_array_elements_text(tc.definition->'marks') as marks 
    from settles.trader_curves tc
    where symbol like '{symbol}' 
    and source like '{source}'
    and date='2021-06-15'  order by symbol,source,date desc
    """
    df = pd.read_sql(sql, con=postgres_prod_engine)
    df[f'{symbol}#{source}'] = df['marks']
    df[f'anchor_spread#{symbol}#{source}'] = df['anchor_spread']
    df.set_index('contracts',inplace=True)
    df.drop(columns=['symbol','epoch_time','marks','date','anchor_spread'], inplace=True)
    if final_df is None:
        final_df = df
    else:
        final_df = final_df.merge(df, how="left", left_index=True, right_index=True)
    settle_date = '2021-06-14'
    url = f"{settles_base_url}getEtrmCurveEval/{r['symbol']}/{settle_date}?eval_type=TimeSpread"
    results = requests.get(url).json()
    settles_df = pd.DataFrame(data=results)
    settles_df.set_index('contract', inplace=True)
    settles_df[f'{symbol}#settles'] = settles_df['static']
    settles_df.drop(columns=['static','dynamic'], inplace=True)
    final_df = final_df.merge(settles_df, how='left', left_index=True, right_index=True)

In [29]:
final_df_sorted = final_df.reindex(sorted(final_df.columns), axis=1)

final_df_sorted.to_csv(r'c:\temp\ts_curves.csv')
# for idx, r in df_prod.iterrows():
#     
#     print(url)
#     results = requests.get(url).json()
#     print(results)
#     df = pd.DataFrame(results['curve'])
#     df['symbol'] = r['symbol']
#     df['updated_at'] = results['updated_at']
#     print(df.head(5))
#     
